In [1]:
import math
import struct
import random
import binascii
from z3 import *

MASK = 0xFFFFFFFF


def xorrayhul64p(state0, state1):
    s0 = state0 & MASK
    s1 = state1 & MASK
    r = (s0 + s1) & MASK
    s1 = (s0 ^ s1) & MASK
    state0 = ((((s0 << 23) | s0 >> (32 - 23)) & MASK) ^ s1 ^ (s1 << 7)) & MASK
    state1 = ((s1 << 18) | s1 >> (32 - 18)) & MASK

    return state0, state1, r

# Symbolic execution of xs128p
# Add the constraint to the solver
# As a note, Z3 has two right shifts — arithmetic right shift and a logical right shift. 
# It uses the arithmetic shift when shifting with the >> operator. must use the logical shift LShR

def z3_xorrayhul64p(slvr, sym_state0, sym_state1, rand_num):
    slvr.add(rand_num == ((sym_state0 + sym_state1) & 0x1FFFFFF))
    sym_state1 = sym_state0 ^ sym_state1
    sym_state0 = (((sym_state0 << 23) | LShR(sym_state0, (32 - 23))) ^ sym_state1 ^ (sym_state1 << 7))
    sym_state1 = ((sym_state1 << 18) | LShR(sym_state1, (32 - 18)))

    return sym_state0, sym_state1


class RNG():
    def __init__(self, seed):
        self.s0 = struct.unpack('>I', seed[:4])[0]
        self.s1 = struct.unpack('>I', seed[4:])[0]
        print(self.s0, self.s1)

    def gen_rand(self):
        self.s0, self.s1, r = xorrayhul64p(self.s0, self.s1)
        return r & 0x1FFFFFF
    

def break_rng():
    # setup symbolic state for xorshiro128+
    ostate0, ostate1 = BitVecs('ostate0 ostate1', 32)
    sym_state0 = ostate0
    sym_state1 = ostate1
    slvr = Solver()
    conditions = []
    
    
    rand_nums = [16254651, 4749029, 16361085]

    # run symbolic xorshiro128+ algorithm for three iterations
    # using the recovered numbers as constraints
    for r in rand_nums:
        sym_state0, sym_state1 = z3_xorrayhul64p(slvr, sym_state0, sym_state1, r)

    if slvr.check() == sat:
        print('SAT')

        # get a solved state
        m = slvr.model()
        state0 = m[ostate0].as_long()
        state1 = m[ostate1].as_long()

        # Print the next set of random numbers
        seed = struct.pack('>I', state0) + struct.pack('>I', state1)
        print(seed)
        rng = RNG(seed)
        for i in range(20):            
            print(rng.gen_rand() % 37)

    else:
        print('UNSAT')


if __name__ == '__main__':
    break_rng()


SAT
b"i'J\xcdI\xd0\xbb\xee"
1764182733 1238416366
33
5
18
36
0
27
16
21
24
0
7
18
34
25
23
29
23
33
29
32
